In [1]:
# import
import torch
import pickle
from tqdm import tqdm
from transformers import BertConfig, BertForPreTraining
# 標準使用ライブラリー
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import shutil
from icecream import ic
from tqdm import tqdm_notebook as tqdm 

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 追記
import json
import datetime
import math
plt.style.use('dark_background')

from torch import nn

from transformers import BertModel

In [2]:
bert = BertModel.from_pretrained("cl-tohoku/bert-base-japanese") # 東北大のものが、mecabでtknzされている

In [3]:
from transformers import BertJapaneseTokenizer # transformersに入ってるものと同じ
tknz = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')


In [4]:
# x_train, x_test = [],[]
# y_train, y_test = [],[]


In [5]:
# train_df = pd.read_csv("../data/train_twitterJSA_data.csv")

# test_df = pd.read_csv("../data/test_twitterJSA_data.csv")

In [6]:
# y_train = train_df["label"].tolist()
# y_test = test_df["label"].tolist()

In [7]:
# for ind in tqdm(range(len(train_df))):
# #     print(ind)
    
#     if type(train_df["text"].iloc[ind]  ) != str:
#         continue
#     tid = tknz.encode(train_df["text"].iloc[ind]) 
    
#     if (len(tid)>512):
#         tid = tid[:512]
#     x_train.append(tid)
#     y_train.append(train_df["label"].iloc[ind])


In [8]:
# for i in range(len(x_train)):
#     if (len(x_train[i])) < 2:
#         print(x_train[i])


In [9]:
# for ind in tqdm(range(len(test_df))):
    
#     if type(test_df["text"].iloc[ind]  ) != str:
#         continue
#     tid = tknz.encode(test_df["text"].iloc[ind]) 
#     if (len(tid)>512):
#         tid = tid[:512]
#     x_test.append(tid)

In [10]:
# for ind in tqdm(range(len(test_df))):
    
#     if type(test_df["text"].iloc[ind] ) != str:
#         continue
    
#     tid = tknz.encode(test_df["text"].iloc[ind]) 
#     if (len(tid)>512):
#         tid = tid[:512]
#     x_test.append(tid)
#     y_test.append(test_df["label"].iloc[ind])
# #     break

In [11]:
# # import pickle

# with open ('x_train.pkl', 'bw') as fw1:
#     pickle.dump( x_train, fw1)
    
# # with open ('y_train.pkl', 'bw') as fw2:
# #     pickle.dump( y_train, fw2)

# # with open ('x_test.pkl', 'bw') as fw3:
# #     pickle.dump( x_test, fw3)

# # with open ('y_test.pkl', 'bw') as fw4:
# #     pickle.dump( y_test, fw4)


In [12]:

with open('x_train.pkl','br') as fr:
    x_train = pickle.load(fr)

with open('y_train.pkl','br') as fr:
    y_train = pickle.load(fr)
    
with open('x_test.pkl','br') as fr:
    x_test = pickle.load(fr)

with open('y_test.pkl','br') as fr:
    y_test = pickle.load(fr)
    
    

In [13]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
class MyDataset(Dataset):
    def __init__(self, xdata, ydata):
        self.data = xdata
        self.label = ydata
    def __len__(self):
        return len(self.label)
    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.label[idx]
        return (x,y)

def my_collate_fn(batch):
    images, targets= list(zip(*batch))
    xs = list(images)
    ys = list(targets)
    return xs, ys

In [14]:
batch_size = 16
dataset = MyDataset(x_train,y_train)
dataloader = DataLoader(dataset, batch_size=batch_size,
                        shuffle=True, collate_fn=my_collate_fn)

In [15]:
class DocCls(nn.Module):
    def __init__(self, bert):
        super(DocCls, self).__init__()
        self.bert = bert
        self.cls = nn.Linear(768, 5)
    
    def forward(self, x1, x2):
        bout = self.bert(input_ids=x1, attention_mask=x2)
        bs = len(bout[0])
        h0 = [bout[0][i][0] for i in range(bs)]
        h0 = torch.stack(h0, dim=0)
        return self.cls(h0)
        
    

In [16]:
from torch import optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


net = DocCls(bert)
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001)

## 学習

In [ ]:
net.train()

for ep in tqdm(range(3)):
    i, lossK=0 ,0.0
    for xs, ys in dataloader:
        xsl, xmsk = [], []
        for k in range(len(xs)):
            tid = xs[k]
            xsl.append(torch.LongTensor(tid))
            xmsk.append(torch.LongTensor([1]*len(tid)))
        
        xsl = pad_sequence(xsl, batch_first = True).to(device)
        xmsk = pad_sequence(xmsk, batch_first = True).to(device)
        outputs = net(xsl, xmsk) 
                        
        dummy_y  =[]

        # 同率１位は、先のほうにする。
                        
        for tmp_y in ys:
            added_flg = 0
            for index, yy in enumerate(tmp_y[1:-1].split(",")):
                if "1" in yy and added_flg == 0:
                    dummy_y.append(index)
                    added_flg = 1

        ys = torch.LongTensor(dummy_y).to(device)
    #         print("out:", out.size(), "y:", y.size())

        loss = criterion(outputs, ys)
        lossK += loss.item()

        if (i % 500 == 0 ):
            print(ep, i, lossK)
            lossK = 0.0

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i+=1
    outfile = "doccls - "+ str(ep) + ".model"
    torch.save(net.state_dict(),outfile)
            

0 0 1.6190744638442993
0 500 238.41799189150333
0 1000 183.2346609607339
0 1500 180.07074964046478
0 2000 168.58060909062624
0 2500 158.8119985088706
0 3000 157.84819432348013
0 3500 159.41249015927315
0 4000 157.2271561101079
0 4500 146.77713668346405
0 5000 149.88358212262392
0 5500 140.57462050765753
0 6000 141.90834756940603



## 評価

In [ ]:
#batch_size = 10
test_dataset = MyDataset(x_test,y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size,
                        shuffle=True, collate_fn=my_collate_fn)

In [ ]:
real_data_num, ok =0, 0
net.eval()

with torch.no_grad():

    for xs, ys in test_dataloader:
        xsl, xmsk = [], []
        for k in range(len(xs)):
            tid = xs[k]
            xsl.append(torch.LongTensor(tid))
            xmsk.append(torch.LongTensor([1]*len(tid)))
        
        xsl = pad_sequence(xsl, batch_first = True).to(device)
        xmsk = pad_sequence(xmsk, batch_first = True).to(device)
        preds = net(xsl, xmsk) 
                        
        dummy_y  =[]

        # 同率１位は、先のほうにする。
        for index_in_batch,tmp_y in enumerate(ys):
            added_flg = 0
            for index, yy in enumerate(tmp_y[1:-1].split(",")):
                if "1" in yy and added_flg == 0:
                    dummy_y.append(index)
                    added_flg = 1
            
            tmp_pred = torch.argmax(preds, dim=0).item()
            if tmp_pred == dummy_y[index_in_batch]:
                ok +=1
            real_data_num +=1
    
    for i in range(len(x_test)):
        x = torch.LongTensor(x_test[i].unsqueeze(0).to(device))
        ans =  net(x)
        ans1 = torch.argmax(ans,dim=1).item()
        if (ans1 == y_test[i]):
            ok += 1
        real_data_num += 1
print(ok, read_data_num, ok/real_data_num)
